## Accessing Relational Databases

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import pathlib

### Acessing [SQLite](https://docs.python.org/3/library/sqlite3.html)  

In [98]:
import sqlite3
conn = sqlite3.connect(os.path.join(outputs,'example.db'))

In [99]:
cur = conn.cursor()

In [100]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[]


In [101]:
# Create table
cur.execute('''CREATE TABLE IF NOT EXISTS stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [102]:
conn = sqlite3.connect(os.path.join(outputs,'example.db'))
cur = conn.cursor()

In [103]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('stocks',)]


In [104]:
t = ('RHAT',) #tuple with just one element
cur.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(cur.fetchone())

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [105]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
cur.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

In [106]:
for row in cur.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


#### Using Pandas

In [107]:
conn = sqlite3.connect(os.path.join(outputs,'example.db'))
df = pd.read_sql_query("SELECT * from stocks ORDER BY price", conn)

# verify that result of SQL query is stored in the dataframe
print(df.head())

conn.close()

         date trans symbol    qty  price
0  2006-01-05   BUY   RHAT  100.0  35.14


Deleting the database file:  

In [108]:
os.remove(os.path.join(outputs,'example.db'))

### Acessing [MySQL](https://pypi.python.org/pypi/PyMySQL)    

In [ ]:
import pymysql

In [ ]:
p = getpass.getpass()
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=p,)# db='example')
cur = conn.cursor()

In [ ]:
cur.execute("show databases;")
for r in cur.fetchall():
   print(r)

In [ ]:
cur.execute("use mysql;")
cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

In [ ]:
cur.execute("CREATE DATABASE IF NOT EXISTS test;")
cur.execute("USE test;")
cur.execute('''CREATE TABLE IF NOT EXISTS test_table 
               (text VARCHAR(200)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

In [ ]:
texts = ['It always seems impossible until its done.',
         'In order to succeed, we must first believe that we can.',
         'Life is 10% what happens to you and 90% how you react to it.',
         'Start where you are. Use what you have. Do what you can.',]
cur.executemany('INSERT INTO test.test_table (text) VALUES (%s)', texts)



In [ ]:
cur.execute("SELECT * FROM test.test_table;")# LIMIT 10;")
for r in cur.fetchall():
   print(r)

#### Using Pandas

In [ ]:
import pandas.io.sql as psql

In [ ]:
df_mysql = psql.read_sql('select * from test.test_table;', con=conn)

In [ ]:
df_mysql.head()

In [ ]:
cur.close()
conn.close()

### Acessing [PostgreSQL](https://www.psycopg.org/docs/)   

In [ ]:
import psycopg2

In [ ]:
p = getpass.getpass()
try:
    conn = psycopg2.connect("dbname='testdb' user='rsouza' host='localhost' password='{}'".format(p))
except:
    print("I am unable to connect to the database")
cur = conn.cursor()

In [ ]:
cur.execute('SELECT * FROM postgres LIMIT 2;')
for r in cur.fetchall():
   print(r)

#### Using Pandas

In [ ]:
p = getpass.getpass()
try:
    conn = psycopg2.connect("dbname='postgres' user='rsouza' host='localhost' password='{}'".format(p))
except:
    print("I am unable to connect to the database")

In [ ]:
df_postgres = psql.read_sql('SELECT * FROM postgres LIMIT 5;', con=conn)
df_postgres.head()